In [ ]:
## Example 1: Read and plot the bathymetry of the SFINCS model
from pathlib import Path
from hydromt_sfincs import SfincsModel

root_folder  = Path(r'../models/sfincs_charleston_hugo').resolve()
sf = SfincsModel(root=root_folder, mode='r')
sf.read()
fig, ax = sf.plot_basemap(
    variable='dep', # plot the bed levels
    bmap='sat', # use satellite background
)

In [ ]:
## Example 2: Inspect the model config
from pathlib import Path
from hydromt_sfincs import SfincsModel

root_folder  = Path(r'../models/sfincs_charleston_hugo').resolve()
sf = SfincsModel(root=root_folder, mode='r')
sf.config  # sfincs.inp file content

In [ ]:
## Example 3: Inspect the model grid
from pathlib import Path
from hydromt_sfincs import SfincsModel

root_folder  = Path(r'../models/sfincs_charleston_hugo').resolve()
sf = SfincsModel(root=root_folder, mode='r')
sf.read()
sf.grid

In [ ]:
## Example 4: Read and plot the model forcing
from pathlib import Path
import sys
# local script imports (replace "../notebooks" with the absolute path to folder containing sfincs_utils.py if needed)
sys.path.append(Path("../notebooks").resolve().as_posix())
from hydromt_sfincs import SfincsModel

root_folder  = Path(r'../models/sfincs_charleston_hugo').resolve()
sf = SfincsModel(root=root_folder, mode='r')
sf.read_forcing()
fig, ax = sf.plot_forcing()

In [ ]:
## Example 5a: Run the model on windows
from pathlib import Path
import sys
# local script imports (replace "../notebooks" with the absolute path to folder containing sfincs_utils.py if needed)
sys.path.append(Path("../notebooks").resolve().as_posix())
from sfincs_utils import run_sfincs

sfincs_exe = Path(r"../software/SFINCS_v2.1.1/sfincs.exe").resolve()
root_folder  = Path(r'../models/sfincs_charleston_hugo').resolve()

run_sfincs(
    Path(root_folder, "sfincs.inp"), # path to the SFINCS model root folder
    run_method="exe", # run SFINCS using the executable
    sfincs_exe=sfincs_exe, # path to the sfincs executable
)

In [ ]:
## Example 5b: Run the model on linux / mac
from pathlib import Path
import sys
# local script imports (replace "../notebooks" with the absolute path to folder containing sfincs_utils.py if needed)
sys.path.append(Path("../notebooks").resolve().as_posix())
from sfincs_utils import run_sfincs

root_folder  = Path(r'../models/sfincs_charleston_hugo').resolve()

# NOTE: make sure docker desktop is installed and running
run_sfincs(
    Path(root_folder, "sfincs.inp"), # path to the SFINCS model root folder
    run_method="docker", # run SFINCS using the docker container
)

In [ ]:
## Example 6: Read the model results and plot the maximum water depth
from pathlib import Path
import numpy as np
from hydromt_sfincs import SfincsModel

root_folder  = Path(r'../models/sfincs_charleston_hugo').resolve()
sf = SfincsModel(root=root_folder, mode='r')
sf.read_results()

# calculate the maximum water depth (hmax) as the difference between 
# the maximum water level (zsmax) and the elevation (dep)
dep = sf.grid["dep"] # average elevation
zsmax = sf.results["zsmax"].max(dim="timemax") # maximum water level
hmax = np.maximum(0, zsmax - dep).round(2).astype(np.float32) # maximum water depth
# Here, we assume that all areas with a negative depth are permanent water bodies
# This is a strong simplification. It is better to use a more sophisticated approach
# to identify permanent water bodies, e.g. based on a water mask or a "tide-only" simulation.
permament_water = dep < 0 
hmax = hmax.where(~permament_water, np.nan) # mask areas with permanent water
# update (geospatial) metadata
hmax.raster.set_crs(dep.raster.crs)
hmax.raster.set_nodata(np.nan)
hmax.name = "max water depth"
hmax.attrs["units"] = "m"
# plot the maximum water depth
fig, ax = sf.plot_basemap(
    # fn_out="max_depth.png",  # uncomment to save figure to mod.root/figs/hmax.png
    variable=hmax,
    plot_bounds=False,
    plot_geoms=False,
    bmap="sat",
    zoomlevel=12,
    figsize=(11, 7),
    vmin=0,  # set minimum value for colorbar
    vmax=1.5,  # set maximum value for colorbar
    cmap="BuPu",
)
ax.set_title("SFINCS maximum flood depth")

# save the maximum water depth to a geo-tiff file
results_folder = Path(sf.root, "../results")
results_folder.mkdir(exist_ok=True) # create the results folder if it does not exist
hmax.raster.to_raster(Path(results_folder, "hmax_hugo.tif"), compress="lzw")

In [ ]:
## Example 7: Read the model results and downscale the water depth to a higher resolution
from pathlib import Path
import hydromt
import numpy as np
from hydromt_sfincs import SfincsModel

root_folder  = Path(r'../models/sfincs_charleston_hugo').resolve()

# We use the subgrid resolution topography to downscale the maximum water level to water depth at the subgrid resolution
# First, we read the subgrid resolution topography and check if the file exists
dep_subgrid_file = Path(root_folder, "subgrid", "dep_subgrid.tif")  
assert dep_subgrid_file.exists(), f"File {dep_subgrid_file} does not exist"
dep_subgrid = hydromt.open_raster(dep_subgrid_file, nodata=np.nan) # elevation at subgrid resolution

# read the SFINCS model results
sf = SfincsModel(root=root_folder, mode='r')
sf.read_results()

# calculate the maximum water depth (hmax) as the difference between 
# the maximum water level (zsmax) and the elevation (dep)
zsmax = sf.results["zsmax"].max(dim="timemax") # maximum water level
zsmax_subgrid = zsmax.raster.reproject_like(dep_subgrid, method="nearest")  # reproject to subgrid resolution
hmax_subgrid = np.maximum(0, zsmax_subgrid - dep_subgrid).round(2).astype(np.float32) # maximum water depth
# Here, we assume that all areas with a negative depth are permanent water bodies
# This is a strong simplification. It is better to use a more sophisticated approach
# to identify permanent water bodies, e.g. based on a water mask or a "tide-only" simulation.
permament_water = dep_subgrid < 0 
hmax_subgrid = hmax_subgrid.where(~permament_water, np.nan) # mask areas with permanent water
# update (geospatial) metadata
hmax_subgrid.raster.set_crs(dep.raster.crs)
hmax_subgrid.raster.set_nodata(np.nan)
hmax_subgrid.name = "max water depth"
hmax_subgrid.attrs["units"] = "m"

# save the maximum water depth to a geo-tiff file
results_folder = Path(sf.root, "../results")
results_folder.mkdir(exist_ok=True) # create the results folder if it does not exist
hmax_subgrid.raster.to_raster(Path(results_folder, "hmax_subgrid_hugo.tif"), compress="lzw")